In [1]:
!pip install -q pogema==1.2.2
!pip install -q stable_baselines3==2.1.0
!pip install -q sb3-contrib==2.1.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 98.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.
tensorflow 2.14.0 requires numpy>=1.23.5, but you have numpy 1.23.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━

In [2]:
import gymnasium as gym
from pogema import GridConfig

from sb3_contrib import RecurrentPPO
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
grid_config = GridConfig(
    size=8,
    density=0.3,
    num_agents=1,
    observation_radius = 3,
    max_episode_steps=128
)

env = gym.make("Pogema-v0",grid_config=grid_config)

/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(


In [4]:

%load_ext autoreload
%autoreload 2
%matplotlib inline

recurrent_ppo = RecurrentPPO(
    "MlpLstmPolicy",
    env,
    verbose=1,


)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
!pip install -q optuna
!pip install -q optuna.pruners
!pip install -q optuna.samplers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement optuna.pruners (from versions: none)
ERROR: No matching distribution found for optuna.pruners
ERROR: Could not find a version that satisfies the requirement optuna.samplers (from versions: none)
ERROR: No matching distribution found for optuna.samplers


In [6]:
from typing import Any
from typing import Dict

import gymnasium
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from sb3_contrib import RecurrentPPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.monitor import Monitor
import torch
import torch.nn as nn


N_TRIALS = 100
N_STARTUP_TRIALS = 5
N_EVALUATIONS = 2
N_TIMESTEPS = int(1.2e5)
EVAL_FREQ = int(N_TIMESTEPS / N_EVALUATIONS)
N_EVAL_EPISODES = 3


ENV_ID = "Pogema-v0"

DEFAULT_HYPERPARAMS = {
    "policy": "MlpLstmPolicy",
    "verbose": 1,
    "env": ENV_ID
}

def sample_recurrentppo_params(trial: optuna.Trial) -> Dict[str, Any]:
    """Sampler for RecurrentPPO hyperparameters."""
    gamma = 1.0 - trial.suggest_float("gamma", 0.0001, 0.1, log=True)
    max_grad_norm = trial.suggest_float("max_grad_norm", 0.3, 5.0, log=True)
    gae_lambda = 1.0 - trial.suggest_float("gae_lambda", 0.001, 0.2, log=True)
    n_steps = 2 ** trial.suggest_int("exponent_n_steps", 3, 10)
    learning_rate = trial.suggest_float("lr", 1e-5, 1, log=True)
    ent_coef = trial.suggest_float("ent_coef", 0.00000001, 0.1, log=True)
    ortho_init = trial.suggest_categorical("ortho_init", [False, True])
    net_arch = trial.suggest_categorical("net_arch", ["tiny", "small"])
    activation_fn = trial.suggest_categorical("activation_fn", ["tanh", "relu"])

    # Display true values.
    trial.set_user_attr("gamma_", gamma)
    trial.set_user_attr("gae_lambda_", gae_lambda)
    trial.set_user_attr("n_steps", n_steps)

    net_arch = [
        {"pi": [64], "vf": [64]} if net_arch == "tiny" else {"pi": [64, 64], "vf": [64, 64]}
    ]

    activation_fn = {"tanh": nn.Tanh, "relu": nn.ReLU}[activation_fn]


    return {
        "n_steps": n_steps,
        "gamma": gamma,
        "gae_lambda": gae_lambda,
        "learning_rate": learning_rate,
        "ent_coef": ent_coef,
        "max_grad_norm": max_grad_norm,
        "policy_kwargs": {
            "net_arch": net_arch,
            "activation_fn": activation_fn,
            "ortho_init": ortho_init,
        },
    }

class TrialEvalCallback(EvalCallback):
    """Callback used for evaluating and reporting a trial."""

    def __init__(
        self,
        eval_env: gymnasium.Env,
        trial: optuna.Trial,
        n_eval_episodes: int = 5,
        eval_freq: int = 10000,
        deterministic: bool = True,
        verbose: int = 0,
    ):
        super().__init__(
            eval_env=eval_env,
            n_eval_episodes=n_eval_episodes,
            eval_freq=eval_freq,
            deterministic=deterministic,
            verbose=verbose,
        )
        self.trial = trial
        self.eval_idx = 0
        self.is_pruned = False

    def _on_step(self) -> bool:
        if self.eval_freq > 0 and self.n_calls % self.eval_freq == 0:
            super()._on_step()
            self.eval_idx += 1
            self.trial.report(self.last_mean_reward, self.eval_idx)
            # Prune trial if need.
            if self.trial.should_prune():
                self.is_pruned = True
                return False
        return True


def objective(trial: optuna.Trial) -> float:
    kwargs = DEFAULT_HYPERPARAMS.copy()
    # Sample hyperparameters.
    kwargs.update(sample_recurrentppo_params(trial))
    # Create the RL model.
    model = RecurrentPPO(**kwargs)
    # Create env used for evaluation.
    eval_env = Monitor(env)
    # Create the callback that will periodically evaluate and report the performance.
    eval_callback = TrialEvalCallback(
        eval_env, trial, n_eval_episodes=N_EVAL_EPISODES, eval_freq=EVAL_FREQ, deterministic=True
    )

    nan_encountered = False
    try:
        model.learn(N_TIMESTEPS, callback=eval_callback)
    except AssertionError as e:
        # Sometimes, random hyperparams can generate NaN.
        print(e)
        nan_encountered = True
    finally:
        # Free memory.
        model.env.close()
        eval_env.close()

    # Tell the optimizer that the trial failed.
    if nan_encountered:
        return float("nan")

    if eval_callback.is_pruned:
        raise optuna.exceptions.TrialPruned()

    return eval_callback.last_mean_reward

if __name__ == "__main__":
    # Set pytorch num threads to 1 for faster training.
    torch.set_num_threads(1)

    sampler = TPESampler(n_startup_trials=N_STARTUP_TRIALS)
    # Do not prune before 1/3 of the max budget is used.
    pruner = MedianPruner(n_startup_trials=N_STARTUP_TRIALS, n_warmup_steps=N_EVALUATIONS // 3)

    study = optuna.create_study(sampler=sampler, pruner=pruner, direction="maximize")
    try:
        study.optimize(objective, n_trials=N_TRIALS, timeout=600)
    except KeyboardInterrupt:
        pass

    print("Number of finished trials: ", len(study.trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    print("  User attrs:")
    for key, value in trial.user_attrs.items():
        print("    {}: {}".format(key, value))

[I 2023-11-10 06:41:50,721] A new study created in memory with name: no-name-65836bc7-d436-47c8-8ea5-2be7ea2564e9
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:29: UserWarning: WARN: It seems a Box observation space is an image but the `dtype` is not `np.uint8`, actual type: float32. If the Box observation space is not an image, we recommend flattening the observation to have only a 1D vector.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/gymnasium/utils/passive_env_checker.py:34: UserWarning: WARN: It seems a Box observation space is an image but the lower and upper bounds are not [0, 255]. Actual lower bound: -1.0, upper bound: 1.0. Generally, CNN policies assume observations are within that range, so you may encounter an issue if the observation values are not.
  logger.warn(
/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/policies.py:460: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you s

Using cpu device
Creating environment from the given name 'Pogema-v0'
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 45.9     |
|    ep_rew_mean     | 0.524    |
| time/              |          |
|    fps             | 241      |
|    iterations      | 1        |
|    time_elapsed    | 4        |
|    total_timesteps | 1024     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 50.2          |
|    ep_rew_mean          | 0.4           |
| time/                   |               |
|    fps                  | 41            |
|    iterations           | 2             |
|    time_elapsed         | 49            |
|    total_timesteps      | 2048          |
| train/                  |               |
|    approx_kl            | 1.3854296e-05 |
|    clip_fraction        | 0

[I 2023-11-10 08:18:26,274] Trial 0 finished with value: 0.0 and parameters: {'gamma': 0.00258565880196216, 'max_grad_norm': 0.7836202013398739, 'gae_lambda': 0.018537464817813452, 'exponent_n_steps': 10, 'lr': 3.273413498949338e-05, 'ent_coef': 0.003489100423222157, 'ortho_init': True, 'net_arch': 'tiny', 'activation_fn': 'relu'}. Best is trial 0 with value: 0.0.


Number of finished trials:  1
Best trial:
  Value:  0.0
  Params: 
    gamma: 0.00258565880196216
    max_grad_norm: 0.7836202013398739
    gae_lambda: 0.018537464817813452
    exponent_n_steps: 10
    lr: 3.273413498949338e-05
    ent_coef: 0.003489100423222157
    ortho_init: True
    net_arch: tiny
    activation_fn: relu
  User attrs:
    gamma_: 0.9974143411980378
    gae_lambda_: 0.9814625351821865
    n_steps: 1024


In [8]:
val_env = gym.make("Pogema-v0",grid_config=grid_config)

val_recurrent_ppo = RecurrentPPO(
     "MlpLstmPolicy",
    val_env,
    verbose=1,
    gamma=0.9974143411980378,
    learning_rate=3.273413498949338e-05,



)

mean_reward, std_reward = evaluate_policy(
    val_recurrent_ppo,
    val_recurrent_ppo.get_env(),
    deterministic=True,
    n_eval_episodes=20,
)

print(f"mean_reward: {mean_reward:.2f} +/- {std_reward:.2f}")

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
mean_reward: 0.10 +/- 0.30


In [9]:
# Train agent and save it
recurrent_ppo.learn(total_timesteps=int(1.2e5))
recurrent_ppo.save("saved/recurrentppo_baseline")

流式输出内容被截断，只能显示最后 5000 行内容。
|    value_loss           | 0.0175     |
----------------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 76.7       |
|    ep_rew_mean          | 0.51       |
| time/                   |            |
|    fps                  | 16         |
|    iterations           | 701        |
|    time_elapsed         | 5486       |
|    total_timesteps      | 89728      |
| train/                  |            |
|    approx_kl            | 0.82803416 |
|    clip_fraction        | 0.252      |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.785     |
|    explained_variance   | -4.91      |
|    learning_rate        | 0.0003     |
|    loss                 | 0.0223     |
|    n_updates            | 7000       |
|    policy_gradient_loss | 0.00131    |
|    value_loss           | 0.00776    |
----------------------------------------
------------------------------

/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/save_util.py:278: UserWarning: Path 'saved' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")


In [11]:
def evaluate_success_rate(model, env, num_episodes=100):
    success_count = 0
    step_array = []
    for _ in range(num_episodes):
        obs = env.reset()

        # Check if observation is a tuple and extract the first element if true.
        if isinstance(obs, tuple):
            obs = obs[0]
        max_step = 128
        steps_taken = 0
        done = truncated = False
        while not done and max_step > 0:
            action, _ = model.predict(obs)
            next_obs, reward, done, truncated, info = env.step(action)
            print(action,max_step,success_count,done)
            max_step -= 1
            steps_taken += 1
            # Check if next_obs is a tuple and extract the first element if true.
            if isinstance(next_obs, tuple):
                next_obs = next_obs[0]
            obs = next_obs

            # Check if agent was successful in that episode.
            if done:
                success_count += 1
                step_array.append(steps_taken)
                break

    success_rate = success_count / num_episodes
    return success_rate, step_array

success_rate,step_array = evaluate_success_rate(recurrent_ppo, env)
print(f"Agent Success Rate: {success_rate * 100:.2f}%")
print(f"steps to termination : {step_array}")

# Calculate the average steps to termination
average_steps = sum(step_array) / len(step_array)
print("Average steps to termination:", average_steps)

流式输出内容被截断，只能显示最后 5000 行内容。
2 53 9 False
2 52 9 False
2 51 9 False
2 50 9 False
2 49 9 False
2 48 9 False
2 47 9 False
2 46 9 False
2 45 9 False
1 44 9 False
2 43 9 False
3 42 9 False
4 41 9 False
2 40 9 False
2 39 9 False
2 38 9 False
1 37 9 False
2 36 9 False
2 35 9 False
2 34 9 False
2 33 9 False
2 32 9 False
2 31 9 False
2 30 9 False
2 29 9 False
4 28 9 False
1 27 9 False
1 26 9 False
4 25 9 False
2 24 9 False
2 23 9 False
2 22 9 False
2 21 9 False
2 20 9 False
2 19 9 False
2 18 9 False
1 17 9 False
4 16 9 False
4 15 9 False
2 14 9 False
3 13 9 False
2 12 9 False
3 11 9 False
3 10 9 False
3 9 9 False
3 8 9 False
3 7 9 False
3 6 9 False
3 5 9 False
3 4 9 False
3 3 9 False
3 2 9 False
3 1 9 False
2 100 9 False
2 99 9 False
2 98 9 False
2 97 9 False
2 96 9 False
2 95 9 False
1 94 9 False
2 93 9 False
2 92 9 False
2 91 9 False
3 90 9 False
2 89 9 False
2 88 9 False
3 87 9 False
2 86 9 False
2 85 9 False
2 84 9 False
2 83 9 False
3 82 9 False
2 81 9 False
2 80 9 False
2 79 9 False
3 78 9

# 新段落